# Neuroidal Model Simulation v1.1
#### Patrick Perrine

Code developed for:

Perrine, P.R. (2023). Neural Tabula Rasa: Foundations for Realistic Memories and Learning. Master's thesis. California Polytechnic State University, San Luis Obispo.

All algorithms described here are primarily derived from:

Valiant, L.G. (2005). Memorization and Association on a Realistic Neural Model. *Neural Computation, 17*, 527–555. https://doi.org/10.1162/0899766053019890

### Install the *graph-tool* library (https://graph-tool.skewed.de/)
The following two code blocks are subject to change based on the library's current hosting location, so a follow up with current information from its website may be required.

In [1]:
%%capture
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

In [2]:
# Run this block if you are on Google Colab, otherwise skip
%%capture
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

### Import other required libraries and seed random number generator

In [3]:
%%capture
import itertools
import numpy as np
from numpy.random import *
import graph_tool.all as gt

In [4]:
global rng
rng = np.random.default_rng(seed=42)

### Define the Neuroidal model's properties
The following configuration is used to create results for the one-step, shared representation model:

* The first six parameters are as described by Valiant (2005).
  * We choose $p = \frac{d}{n-1}$ due to the low $n$ values of tested here.
* The $r_{approx}$ parameter is manually determined beforehand based on the previous parameters, and is used to generate the random, initial memories in a fixed size pursuant to the relations defined in Valiant (2005).
* The $I$ parameter is what determines when an instance of interference is recognized.
* The $L$ parameter specifies how many initial memories to grant the model before calling JOIN.
* The $F$ parameter determines the level of interference tolerance to be checked at each iteration of JOIN.


In [5]:
n = 1000
d = 64
p = d / n
t = 1
k = 8
k_adj = 1.2

r_approx = 45

I = 2
L = 100
F = 0.25

## Generate the graph model

### Create an empty graph

In [6]:
global g
g = gt.Graph(directed=True)

### Populate our graph with $N$ vertices

In [7]:
%%capture
g.add_vertex(n)

### Define the mode of each neuron of the graph

In [8]:
mode_T = g.new_vertex_property("int")
mode_q = g.new_vertex_property("int")
mode_f = g.new_vertex_property("bool")

### Initialize the mode of each neuron

In [9]:
mode_T.a = t
mode_q.a = 1
mode_f.a = False

### Populate our graph with synapses

In [10]:
all_edges = itertools.permutations(range(n), 2)
for e in all_edges:
    if rng.random() < p:
         g.add_edge(*e)

### Define the mode of each synapse

In [11]:
mode_qq = g.new_vertex_property("int")
mode_w = g.new_edge_property("double")

### Initialize the mode of each synapse

In [12]:
mode_qq.a = 1
mode_w.a = t / (k_adj * k)

## Vicinal Algorithms for the JOIN algorithm

### Weight summation functions

In [13]:
def sum_weights(v_i):
    w_i = 0
    for e_ji in g.iter_in_edges(v_i):
        if mode_f[e_ji[0]] == True:
            w_i += mode_w[e_ji]
    return w_i

Equivalent to the function above, but leverages optimization offered by graph-tool and NumPy to negate the costly for loop.

In [14]:
def fast_sum_weights(v_i):
    W = g.get_in_edges(v_i, [mode_w])[:,2]
    F = np.array(g.get_in_neighbors(v_i, [mode_f])[:,1], dtype=bool)
    return W[F].sum()

### Neuron and Synapse Updates

In [15]:
def _delta(v_i, w_i):
    if w_i > mode_T[v_i]:
        mode_q[v_i] = 2
        mode_f[v_i] = True

In [16]:
def _lambda(v_i, w_i, e_ji, f_j):
    if f_j == 1:
        mode_qq[e_ji] = 2

### Overall Graph Update

For convenience, we check for eligible $C$ nodes during the weight updates for each neuron of the graph. We also allow the skipping of updating the edges of the graph for our use of JOIN in the simulation, as edge updates are not needed in our case.

Also, we would normally allow a neuron to fire when its threshold is surpassed by its incoming edge weights (a threshold transition). However, we reverse such firing here due to it affecting the search for memory $C$ when other nodes will later be checked.

In [17]:
def update_graph(update_edges=False):
    C = []
    for v_i in g.iter_vertices():
        w_i = fast_sum_weights(v_i)
        _delta(v_i, w_i)
        if mode_q[v_i] == 2:
            C.append(v_i)
            mode_f[v_i] = False
        if update_edges:
            for e_ji in g.iter_in_edges(v_i):
                f_j = mode_f[e_ji[0]]
                _lambda(v_i, w_i, e_ji, f_j)
    mode_q.a = 1
    mode_qq.a = 1
    mode_f.a = False
    return C

## The JOIN algorithm
This implements the *one-step* variant of JOIN for *shared representations* as
defined in Valiant (2005).

In [18]:
def JOIN_one_step_shared(A, B):
    for v in A + B:
        mode_f[v] = True
    return update_graph(update_edges=False)

## Interference check function

In [19]:
def interference_check(S, A_i, B_i, C):
    sum = 0
    for D_i in range(len(S)):
        if D_i != A_i and D_i != B_i:
            D = S[D_i]
            if len(set(C) & set(D)) > (len(D) / I):
                sum += I
    return sum

## Simulation

### Verbose print functions for simulation updates

In [20]:
def print_join_update(S_length, H, H_if, total_if, m_size):
    print("Current Total Memories:", S_length)
    print("Current Interference Rate:", round(H_if/H, 3))
    print("Averaged Interference Rate:", round(total_if/S_length, 3))
    print("Averaged Size of Memories Created:", round(m_size/H, 2), "\n\n")

In [21]:
def print_halt_msg(S_length, H, H_if, total_if):
    print("----\n\n")
    print("Given: n=", n, "d=", d, "k=", k, "k_adj=", k_adj, "r_approx=",
           r_approx, "START_MEM=", L)
    print("Halting Memory Formation at", int(F*100), "% Total Interference")
    print("Total Averaged Interference Rate:", round(total_if/S_length, 3))
    print("Capacity:", L, "Initial Memories +" ,S_length-L,"JOIN Memories.")

In [22]:
def print_memorized_msg(S_length):
    print("Given: n=", n, "d=", d, "k=", k, "k_adj=", k_adj, "r_approx=",
           r_approx, "START_MEM=", L)
    print("Memorized all combinations of", L,"memory connectives.")
    print("Contains:", L, "Initial Memories +" ,S_length-L,"JOIN Memories.")

### Call JOIN on all possible pairs of memories until the interference threshold is reached
Assuming that the interference faulting proportion will be reached and this code's execution will halt, we will have reached the model's final capacity. Otherwise, we had not specified a large enough $L$ to give the model enough information to memorize disjunctions from, leading to a "total memorization," which is not very useful for measuring capacity.

Note that the $H$ parameter determines how many memories to JOIN in one "batch" of simulation.

In [23]:
def one_step_shared_join_simulation(H, verbose=True):
    m = 0
    H_if = 0
    m_size = 0
    total_if = 0
    init_pairs = itertools.combinations(range(L), 2)
    S = [rng.choice(np.arange(0,n-1), size=r_approx)
         for _ in range(L)]

    for A_i,B_i in init_pairs:
        A = list(S[A_i])
        B = list(S[B_i])
        C = JOIN_one_step_shared(A, B)
        C_if = interference_check(S, A_i, B_i, C)

        m += 1
        S.append(C)
        m_size += len(C)

        if m % H == 0:
            if verbose:
                print_join_update(len(S), H, H_if, total_if, m_size)
            H_if = 0
            m_size = 0

        if C_if > 0:
            H_if += C_if
            total_if += C_if
            if total_if/len(S) > F:
              if verbose:
                  print_halt_msg(len(S), H, H_if, total_if)
              return S

    if verbose:
        print_memorized_msg(len(S))
    return S

In [24]:
memory_bank = one_step_shared_join_simulation(H=1000, verbose=True)

Current Total Memories: 1100
Current Interference Rate: 0.098
Averaged Interference Rate: 0.089
Averaged Size of Memories Created: 43.45 


Current Total Memories: 2100
Current Interference Rate: 0.264
Averaged Interference Rate: 0.172
Averaged Size of Memories Created: 41.56 


Current Total Memories: 3100
Current Interference Rate: 0.386
Averaged Interference Rate: 0.241
Averaged Size of Memories Created: 43.15 


----


Given: n= 1000 d= 64 k= 8 k_adj= 1.2 r_approx= 45 START_MEM= 100
Halting Memory Formation at 25 % Total Interference
Total Averaged Interference Rate: 0.251
Capacity: 100 Initial Memories + 3657 JOIN Memories.


### Comparison with the exact capacity for the *disjoint representation* model

In [25]:
print("Exact capacity in disjoint representation:", (int)(n / r_approx))

Exact capacity in disjoint representation: 22
